In [1]:
# installing appropiate libraries and packages

%pip install pandas
%pip install numpy
%pip install rouge-score
%pip install matplotlib
%pip install sumy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 97 kB 4.8 MB/s eta 0:00:011
     |████████████████████████████████| 10.1 MB 9.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rouge_score import rouge_scorer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer


In [3]:
# loading full dataset in
movies = pd.read_csv('rotten_tomatoes_movies.csv')
reviews = pd.read_csv('rotten_tomatoes_critic_reviews.csv')

In [4]:
# dropping movies that don't have ground truth
filteredMovies = movies.dropna(subset = "critics_consensus")

In [40]:
# parameter: how many movies to conduct the analysis for (CHANGE THIS)
to_keep = 1000

furtherFilteredMovies = filteredMovies[:to_keep]

In [41]:
movies_to_keep = furtherFilteredMovies[['rotten_tomatoes_link']]

# data table for each critic review that has ground truth
combined_data = movies_to_keep.merge(reviews, how = 'left')

In [42]:
combined_data.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [43]:
# dropping data that don't have review values
filtered_combined_data = combined_data.dropna(subset = "review_content")

In [44]:
# grouping critic reviews by each movie and then appending together each review for a given given
filtered_appended_reviews = \
pd.DataFrame(filtered_combined_data.groupby("rotten_tomatoes_link")['review_content'].apply(lambda x: ','.join(x))).reset_index()

In [45]:
# pulling together a table table that represents each movie and the pre_hypothesis for the summary(joining together of all the reviews for a movie)
final_data = \
furtherFilteredMovies.merge(filtered_appended_reviews, on = 'rotten_tomatoes_link')[['movie_title', 'critics_consensus', 'review_content']]

In [46]:
final_data.head()

,movie_title,critics_consensus,review_content
0,Percy Jackson & the Olympians: The Lightning T...,Though it may seem like just another Harry Pot...,A fantasy adventure that fuses Greek mythology...
1,Please Give,Nicole Holofcener's newest might seem slight i...,"Like Holofcener's previous pictures, Please Gi..."
2,10,Blake Edwards' bawdy comedy may not score a pe...,10 (1979) is known for its numerical rating sy...
3,12 Angry Men (Twelve Angry Men),Sidney Lumet's feature debut is a superbly wri...,"A film with texture, humour and relevance at a..."
4,"20,000 Leagues Under The Sea","One of Disney's finest live-action adventures,...",[The] embodiment of Disney at his best -- fami...


In [12]:
# the human-summarizations of all the reviews for a movie
ground_truth = list(final_data['critics_consensus'])

In [13]:
# the appended reviews for a given movie
appended_reviews_pre_hypotheses = list(final_data['review_content'])

In [28]:
final_hypothesis_textrank = []
final_hypothesis_lexrank = []
final_hypothesis_lsa = []

summarizer_textrank = TextRankSummarizer()
summarizer_lexrank = LexRankSummarizer()
summarizer_lsa = LsaSummarizer()

for each_movie_appended_reviews_hypothesis in appended_reviews_pre_hypotheses:
    parser = PlaintextParser.from_string(each_movie_appended_reviews_hypothesis,Tokenizer("english"))

    # TextRank
    summary_lexrank = summarizer_lexrank(parser.document,2)
    summary_textrank = [str(elt) for elt in summary_textrank]
    summary_textrank = " ".join(summary_textrank)
    final_hypothesis_textrank.append(summary_textrank)

    # LexRank
    summary_lexrank = summarizer_lexrank(parser.document,2)
    summary_lexrank = [str(elt) for elt in summary_lexrank]
    summary_lexrank = " ".join(summary_lexrank)
    final_hypothesis_lexrank.append(summary_lexrank)

    # LSA
    summary_lsa = summarizer_lsa(parser.document,2)
    summary_lsa = [str(elt) for elt in summary_lsa]
    summary_lsa = " ".join(summary_lsa)
    final_hypothesis_lsa.append(summary_lsa)


In [36]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)

#running the ROUGE evaluation for certain subset of ground truth and hypothesis pairs
num_of_iterations = 100
all_rouges_1_scores_textrank = []
all_rouges_2_scores_textrank = []

all_rouges_1_scores_lexrank = []
all_rouges_2_scores_lexrank = []

all_rouges_1_scores_lsa = []
all_rouges_2_scores_lsa = []

for i in range(num_of_iterations):
    all_scores_obj_textrank = scorer.score(final_hypothesis_textrank[i], ground_truth[i])
    all_rouges_1_scores_textrank.append(all_scores_obj_textrank["rouge1"])
    all_rouges_2_scores_textrank.append(all_scores_obj_textrank["rouge2"])

    all_scores_obj_lexrank = scorer.score(final_hypothesis_lexrank[i], ground_truth[i])
    all_rouges_1_scores_lexrank.append(all_scores_obj_lexrank["rouge1"])
    all_rouges_2_scores_lexrank.append(all_scores_obj_lexrank["rouge2"])

    all_scores_obj_lsa = scorer.score(final_hypothesis_lsa[i], ground_truth[i])
    all_rouges_1_scores_lsa.append(all_scores_obj_lsa["rouge1"])
    all_rouges_2_scores_lsa.append(all_scores_obj_lsa["rouge2"])

In [37]:
# building up metric results for Text Rank Algorithm

# Rouge 1
rouge_1_precisions_textrank = []
rouge_1_recall_textrank = []
rouge_1_f1_textrank = []
for each_rouge_1_score in all_rouges_1_scores_textrank:
    rouge_1_precisions_textrank.append(each_rouge_1_score.precision)
    rouge_1_recall_textrank.append(each_rouge_1_score.recall)
    rouge_1_f1_textrank.append(each_rouge_1_score.fmeasure)

avg_rouge_1_precision_score_textrank = np.average(rouge_1_precisions_textrank)
avg_rouge_1_recall_score_textrank = np.average(rouge_1_recall_textrank)
avg_rouge_1_f1_score_textrank = np.average(rouge_1_f1_textrank)

# Rouge 2
rouge_2_precisions_textrank = []
rouge_2_recall_textrank = []
rouge_2_f1_textrank = []
for each_rouge_2_score in all_rouges_2_scores_textrank:
    rouge_2_precisions_textrank.append(each_rouge_2_score.precision)
    rouge_2_recall_textrank.append(each_rouge_2_score.recall)
    rouge_2_f1_textrank.append(each_rouge_2_score.fmeasure)

avg_rouge_2_precision_score_textrank = np.average(rouge_2_precisions_textrank)
avg_rouge_2_recall_score_textrank = np.average(rouge_2_recall_textrank)
avg_rouge_2_f1_score_textrank = np.average(rouge_2_f1_textrank)

print("TEXT RANK ALGORITHM METRICS")
print("------")
print("Rouge 1:")
print(f'The average precision value for the rouge 1 scores is {avg_rouge_1_precision_score_textrank}')
print(f'The average recall value for the rouge 1 scores is {avg_rouge_1_recall_score_textrank}')
print(f'The average f1 value for the rouge 1 scores is {avg_rouge_1_f1_score_textrank}')
print("")
print("Rouge 2:")
print(f'The average precision value for the rouge 2 scores is {avg_rouge_2_precision_score_textrank}')
print(f'The average recall value for the rouge 2 scores is {avg_rouge_2_recall_score_textrank}')
print(f'The average f1 value for the rouge 2 scores is {avg_rouge_2_f1_score_textrank}')

TEXT RANK ALGORITHM METRICS
------
Rouge 1:
The average precision value for the rouge 1 scores is 0.6756509057229748
The average recall value for the rouge 1 scores is 0.03072333815284916
The average f1 value for the rouge 1 scores is 0.05791317688811271

Rouge 2:
The average precision value for the rouge 2 scores is 0.207613474483758
The average recall value for the rouge 2 scores is 0.008329307502567218
The average f1 value for the rouge 2 scores is 0.01577995648762225


In [38]:
# building up metric results for Lex Rank Algorithm

# Rouge 1
rouge_1_precisions_lexrank = []
rouge_1_recall_lexrank = []
rouge_1_f1_lexrank = []
for each_rouge_1_score in all_rouges_1_scores_lexrank:
    rouge_1_precisions_lexrank.append(each_rouge_1_score.precision)
    rouge_1_recall_lexrank.append(each_rouge_1_score.recall)
    rouge_1_f1_lexrank.append(each_rouge_1_score.fmeasure)

avg_rouge_1_precision_score_lexrank = np.average(rouge_1_precisions_lexrank)
avg_rouge_1_recall_score_lexrank = np.average(rouge_1_recall_lexrank)
avg_rouge_1_f1_score_lexrank = np.average(rouge_1_f1_lexrank)

# Rouge 2
rouge_2_precisions_lexrank = []
rouge_2_recall_lexrank = []
rouge_2_f1_lexrank = []
for each_rouge_2_score in all_rouges_2_scores_lexrank:
    rouge_2_precisions_lexrank.append(each_rouge_2_score.precision)
    rouge_2_recall_lexrank.append(each_rouge_2_score.recall)
    rouge_2_f1_lexrank.append(each_rouge_2_score.fmeasure)

avg_rouge_2_precision_score_lexrank = np.average(rouge_2_precisions_lexrank)
avg_rouge_2_recall_score_lexrank = np.average(rouge_2_recall_lexrank)
avg_rouge_2_f1_score_lexrank = np.average(rouge_2_f1_lexrank)

print("LEX RANK ALGORITHM METRICS")
print("------")
print("Rouge 1:")
print(f'The average precision value for the rouge 1 scores is {avg_rouge_1_precision_score_lexrank}')
print(f'The average recall value for the rouge 1 scores is {avg_rouge_1_recall_score_lexrank}')
print(f'The average f1 value for the rouge 1 scores is {avg_rouge_1_f1_score_lexrank}')
print("")
print("Rouge 2:")
print(f'The average precision value for the rouge 2 scores is {avg_rouge_2_precision_score_lexrank}')
print(f'The average recall value for the rouge 2 scores is {avg_rouge_2_recall_score_lexrank}')
print(f'The average f1 value for the rouge 2 scores is {avg_rouge_2_f1_score_lexrank}')

LEX RANK ALGORITHM METRICS
------
Rouge 1:
The average precision value for the rouge 1 scores is 0.6505254436967709
The average recall value for the rouge 1 scores is 0.03882336350928963
The average f1 value for the rouge 1 scores is 0.07080522339232535

Rouge 2:
The average precision value for the rouge 2 scores is 0.1875147042404839
The average recall value for the rouge 2 scores is 0.010020281580821461
The average f1 value for the rouge 2 scores is 0.018394956239432422


In [39]:
# building up metric results for LSA Algorithm

# Rouge 1
rouge_1_precisions_lsa = []
rouge_1_recall_lsa = []
rouge_1_f1_lsa = []
for each_rouge_1_score in all_rouges_1_scores_lsa:
    rouge_1_precisions_lsa.append(each_rouge_1_score.precision)
    rouge_1_recall_lsa.append(each_rouge_1_score.recall)
    rouge_1_f1_lsa.append(each_rouge_1_score.fmeasure)

avg_rouge_1_precision_score_lsa = np.average(rouge_1_precisions_lsa)
avg_rouge_1_recall_score_lsa = np.average(rouge_1_recall_lsa)
avg_rouge_1_f1_score_lsa = np.average(rouge_1_f1_lsa)

# Rouge 2
rouge_2_precisions_lsa = []
rouge_2_recall_lsa = []
rouge_2_f1_lsa = []
for each_rouge_2_score in all_rouges_2_scores_lsa:
    rouge_2_precisions_lsa.append(each_rouge_2_score.precision)
    rouge_2_recall_lsa.append(each_rouge_2_score.recall)
    rouge_2_f1_lsa.append(each_rouge_2_score.fmeasure)

avg_rouge_2_precision_score_lsa = np.average(rouge_2_precisions_lsa)
avg_rouge_2_recall_score_lsa = np.average(rouge_2_recall_lsa)
avg_rouge_2_f1_score_lsa = np.average(rouge_2_f1_lsa)

print("LSA ALGORITHM METRICS")
print("------")
print("Rouge 1:")
print(f'The average precision value for the rouge 1 scores is {avg_rouge_1_precision_score_lsa}')
print(f'The average recall value for the rouge 1 scores is {avg_rouge_1_recall_score_lsa}')
print(f'The average f1 value for the rouge 1 scores is {avg_rouge_1_f1_score_lsa}')
print("")
print("Rouge 2:")
print(f'The average precision value for the rouge 2 scores is {avg_rouge_2_precision_score_lsa}')
print(f'The average recall value for the rouge 2 scores is {avg_rouge_2_recall_score_lsa}')
print(f'The average f1 value for the rouge 2 scores is {avg_rouge_2_f1_score_lsa}')

LSA ALGORITHM METRICS
------
Rouge 1:
The average precision value for the rouge 1 scores is 0.5485955792813172
The average recall value for the rouge 1 scores is 0.05929854300898421
The average f1 value for the rouge 1 scores is 0.10102388173633196

Rouge 2:
The average precision value for the rouge 2 scores is 0.12889357037439042
The average recall value for the rouge 2 scores is 0.01099356222196345
The average f1 value for the rouge 2 scores is 0.0191179406094523
